In [2]:
!!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib python-docx pandas python-pptx langchain_community


['Collecting google-api-python-client',
 '  Downloading google_api_python_client-2.156.0-py2.py3-none-any.whl.metadata (6.7 kB)',
 'Collecting google-auth-httplib2',
 '  Downloading google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)',
 'Collecting google-auth-oauthlib',
 '  Downloading google_auth_oauthlib-1.2.1-py2.py3-none-any.whl.metadata (2.7 kB)',
 'Requirement already satisfied: python-docx in c:\\users\\dell\\anaconda3\\lib\\site-packages (1.1.2)',
 'Requirement already satisfied: pandas in c:\\users\\dell\\anaconda3\\lib\\site-packages (2.1.4)',
 'Collecting python-pptx',
 '  Downloading python_pptx-1.0.2-py3-none-any.whl.metadata (2.5 kB)',
 'Collecting langchain_community',
 '  Downloading langchain_community-0.3.13-py3-none-any.whl.metadata (2.9 kB)',
 'Collecting httplib2<1.dev0,>=0.19.0 (from google-api-python-client)',
 '  Downloading httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)',
 'Collecting google-auth!=2.24.0,!=2.25.0,<3.0.0.dev0,>=1.32.0 (from go

In [13]:
!pip install pypdf

   ---------------------------------------- 0.0/298.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/298.0 kB ? eta -:--:--
   - -------------------------------------- 10.2/298.0 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/298.0 kB 262.6 kB/s eta 0:00:02
   ----- --------------------------------- 41.0/298.0 kB 326.8 kB/s eta 0:00:01
   ------------ -------------------------- 92.2/298.0 kB 476.3 kB/s eta 0:00:01
   -------------- ----------------------- 112.6/298.0 kB 467.6 kB/s eta 0:00:01
   --------------- ---------------------- 122.9/298.0 kB 479.3 kB/s eta 0:00:01
   ------------------------ ------------- 194.6/298.0 kB 588.9 kB/s eta 0:00:01
   ------------------------------- ------ 245.8/298.0 kB 654.2 kB/s eta 0:00:01
   -------------------------------------- 298.0/298.0 kB 708.7 kB/s eta 0:00:00


In [ ]:
import os, re
import io
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.errors import HttpError
from docx import Document
import pandas as pd
from pptx import Presentation
from langchain_community.document_loaders import PyPDFLoader

SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]

def clean_text(text):
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Fix words broken by line breaks or formatting
    # Matches a lowercase/uppercase word followed by a line break without a space
    text = re.sub(r'(?<=[a-zA-Z])(?=[A-Z])', ' ', text)  # Add space between words when they are stuck together    
    # Fix punctuation followed by words with no space
    text = re.sub(r'(?<=[.,?!;])(?=[a-zA-Z])', ' ', text)  # Add space after punctuation if missing
    # Standardize newlines for better formatting
    text = re.sub(r'\.\s+', '.\n', text)  # Add newlines after sentences
    text = re.sub(r'(?<=:)\s+', '\n', text)  # Add newlines after colons
    # Additional cleanup (if needed)    text = text.strip()  # Remove leading and trailing whitespace
    return text

def getCredentials():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    return creds

def getFilesInFolder(folder_id):
    """Fetches files from the specified Google Drive folder."""
    files = []
    try:
        service = build("drive", "v3", credentials=getCredentials())
        page_token = None
        while True:
            # Call the Drive v3 API
            results = (
                service.files()
                .list(q=f"'{folder_id}' in parents",  # Specify folder_id
                      spaces="drive",
                      fields="nextPageToken, files(id, name)",
                      pageToken=page_token)
                .execute()
            )
            items = results.get("files", [])
            for item in items:
                files.append(item)
            page_token = results.get("nextPageToken", None)
            if not page_token:
                break
    except HttpError as error:
        print(f"An error occurred: {error}")
    return files

def download_file(service, file_id, file_name):
    """Download a file from Google Drive."""
    request = service.files().get_media(fileId=file_id)
    fh = io.FileIO(file_name, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(f"Download {int(status.progress() * 100)}%.")
    return file_name

def load_hidden_documents_from_drive(files):
    all_texts = []
    for file in files:
        file_name = file['name']
        file_id = file['id']
        
        # Download the file from Google Drive
        print(f"Downloading file: {file_name}")
        downloaded_file_name = download_file(service, file_id, file_name)
        
        # Read the downloaded file content and extract text
        if downloaded_file_name.endswith(".pdf"):
            loader = PyPDFLoader(downloaded_file_name)
            all_texts.extend([page.page_content for page in loader.load_and_split()])
        elif downloaded_file_name.endswith(".docx"):
            doc = Document(downloaded_file_name)
            all_texts.append("\n".join(p.text for p in doc.paragraphs))
        elif downloaded_file_name.endswith(".txt"):
            with open(downloaded_file_name, "r", encoding="utf-8") as f:
                all_texts.append(f.read())
        elif downloaded_file_name.endswith((".xlsx", ".xls")):
            all_texts.append(pd.read_excel(downloaded_file_name).to_string(index=False))
        elif downloaded_file_name.endswith(".pptx"):
            presentation = Presentation(downloaded_file_name)
            all_texts.append("\n".join(shape.text for slide in presentation.slides for shape in slide.shapes if shape.has_text_frame))
        
        # Delete the file after processing
        os.remove(downloaded_file_name)
    
    return all_texts

if __name__ == "__main__":
    # Specify your folder ID here
    folder_id = "1ncoIEIw_P2__rQHBdbyLjkQbOHNMmQi6"
    
    # Fetch the files from the specified folder
    service = build("drive", "v3", credentials=getCredentials())
    files = getFilesInFolder(folder_id)
    
    # Process the files to extract text and build all_texts
    all_texts_list = load_hidden_documents_from_drive(files)
    all_texts = clean_text(" ".join(all_texts_list))  # Join the list into a single string
    print("Collected all texts from files:")
    print(all_texts)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1048383435215-1ot97aeh8qauc6mufmbfpvei4m0mkn5c.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64426%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=g6lCVoyOy6oo7hD9huecBCiTs2fFZA&access_type=offline
